**Demo for `teneva.core_jax.act_one`**

---

This module contains the basic operations with one TT-tensor (Y), including "copy", "get", "sum", etc.

## Loading and importing modules

In [1]:
import jax
import jax.numpy as np
import teneva as teneva_base
import teneva.core_jax as teneva
from time import perf_counter as tpc
rng = jax.random.PRNGKey(42)

## Function `copy`

Return a copy of the given TT-tensor.

In [2]:
# 10-dim random TT-tensor with mode size 4 and TT-rank 12:
rng, key = jax.random.split(rng)
Y = teneva.rand(10, 4, 12, key)

Z = teneva.copy(Y) # The copy of Y  

print(Y[2][1, 2, 0])
print(Z[2][1, 2, 0])

0.798532
0.798532


## Function `get`

Compute the element with multiindex "k" from the TT-tensor "Y".

In [3]:
d = 5  # Dimension of the tensor
n = 4  # Mode size of the tensor
r = 2  # Rank of the tensor

# Construct d-dim full array:
t = np.arange(2**d) # Tensor will be 2^d
Y0 = np.cos(t).reshape([2] * d, order='F')

# Compute TT-tensor from Y0 by TT-SVD:  
Y1 = teneva.svd(Y0, r)

# Print the TT-tensor:
teneva.show(Y1)

# Select some tensor element and compute the value:
k = np.array([0, 1, 0, 1, 0])
y1 = teneva.get(Y1, k)

# Compute the same element of the original tensor:
y0 = Y0[tuple(k)]

# Compare original tensor and reconstructed tensor:
e = np.abs(y1-y0)
print(f'Error : {e:7.1e}')

TT-tensor     5D (rank =     2)
Error : 4.8e-07


## Function `get_many`

In [4]:
d = 5  # Dimension of the tensor
n = 4  # Mode size of the tensor
r = 2  # Rank of the tensor

# Construct d-dim full array:
t = np.arange(2**d) # Tensor will be 2^d
Y0 = np.cos(t).reshape([2] * d, order='F')

# Compute TT-tensor from Y0 by TT-SVD:  
Y1 = teneva.svd(Y0, r)

# Print the TT-tensor:
teneva.show(Y1)

# Select some tensor element and compute the value:
K = np.array([
    [0, 1, 0, 1, 0],
    [0, 0, 0, 0, 0],
    [1, 1, 1, 1, 1],
])
y1 = teneva.get_many(Y1, K)

# Compute the same element of the original tensor:
y0 = np.array([Y0[tuple(k)] for k in K])

# Compare original tensor and reconstructed tensor:
e = np.max(np.abs(y1-y0))
print(f'Error : {e:7.1e}')

TT-tensor     5D (rank =     2)
Error : 6.6e-07


We can compare the calculation time using the base function ("get") with "jax.vmap" and the function "get_many":

In [5]:
d = 1000   # Dimension of the tensor
n = 100    # Mode size of the tensor
r = 10     # Rank of the tensor

rng, key = jax.random.split(rng)
Y = teneva.rand(d, n, r, key)

get1 = jax.jit(jax.vmap(teneva.get, (None, 0)))
get2 = jax.jit(teneva.get_many)

for m in [1, 1.E+1, 1.E+2, 1.E+3, 1.E+4, 1.E+5, 1.E+6]:
    I = np.array(teneva_base.sample_lhs([n]*d, int(m))) # TODO

    t1 = tpc()
    y1 = get1(Y, I)
    t1 = tpc() - t1

    t2 = tpc()
    y2 = get2(Y, I)
    t2 = tpc() - t2

    print(f'm: {m:-7.1e} | T1 : {t1:-8.4f} | T2 : {t2:-8.4f}')

m: 1.0e+00 | T1 :   0.0659 | T2 :   0.0654
m: 1.0e+01 | T1 :   0.0873 | T2 :   0.0859
m: 1.0e+02 | T1 :   0.0953 | T2 :   0.0974
m: 1.0e+03 | T1 :   0.2014 | T2 :   0.1640
m: 1.0e+04 | T1 :   0.4863 | T2 :   0.4746
m: 1.0e+05 | T1 :   8.0010 | T2 :   7.9864
m: 1.0e+06 | T1 :  92.5100 | T2 :  90.8101


---